# Classes and objects - an introduction

In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
class Fish():
    pass

In [3]:
a_fish = Fish()
another_fish = Fish()

a_fish == another_fish

False

In [4]:
a_fish = Fish()
a_second_ref_to_first_fish = a_fish

a_fish == a_second_ref_to_first_fish

True

In [5]:
class Fish():
    pass

In [8]:
class Fish():
    def __init__(self, age=7):        
        self.age = age

In [13]:
a_fish = Fish(age=6)
another_fish = Fish(age=8)
print("age of a fish: {}".format(a_fish.age))
print("age of another fish: {}".format(another_fish.age))

age of a fish: 6
age of another fish: 8


In [14]:
another_fish.age = 9
print("age of a fish: {}".format(a_fish.age))
print("age of another fish: {}".format(another_fish.age))

age of a fish: 6
age of another fish: 9


In [98]:
a_second_ref_to_first_fish = a_fish
a_second_ref_to_first_fish.age = 10
print("age of a fish: {}".format(a_fish.age))

age of a fish: 10


In [99]:
another_fish.age = 9
print("age of a fish: {}".format(a_fish.age))
print("age of another fish: {}".format(another_fish.age))

age of a fish: 10
age of another fish: 9


For reasons that will be clearer later, sometimes we might want a function to ask for the value of specific attribute of an object:

In [100]:
getattr(a_fish, "age")

10

In [102]:
getattr(a_fish, "color", "black")

'black'

## What are classes useful for?

Imagine we want to simulate trajectories from three fish. Each fish will have coordinates, position, and bout probability. How do we keep track of all these parameters for all fish?
Of course we could make arrays keeping track of all variables:

In [ ]:
fish_x_pos = np.zeros(3)
fish_y_pos = np.zeros(3)
fish_vel = np.array([5, 10, 3])

def update_positions(fish_x_pos, fish_x_pos, fish_vel):
    ...

In such circumstances, classes can be very useful to keep together attributes that are referring to the same thing (object, a fish in this case) and operate on them. For example, for a moving fish:

In [22]:
class Fish():
    def __init__(self, age, start_pos_x=0, start_pos_y=0, max_speed=10):
        
        self.age = age
        self.pos_x = start_pos_x
        self.pos_y = start_pos_y
        
        self.max_speed = 10

    def bout(self):
        self.pos_x += np.random.randint(-self.max_speed, self.max_speed)
        self.pos_y += np.random.randint(-self.max_speed, self.max_speed)
        
    def print_pos(self):
        print(self.pos_x, self.pos_y)
        
    def reset_pos(self, pos_x, pos_y):
        self.pos_x = pos_x
        self.pos_y = pos_y

In [23]:
a_fish = Fish(8)

# Inheritance

In [26]:
class FishLarvae(Fish):
    def is_stupid():  # Note: we don't have the self here! This we can do, because we don't use it.
        print("Yes!")

class FishAdult(Fish):
    def is_stupid():
        print("No!")
        
    def do_something_smart(self):
        print("something smart")

In [33]:
a_larvae_fish = FishAdult(age=5)
an_adult_fish = FishAdult(age=23)

In [29]:
print(isinstance(an_adult_fish, Fish))
print(isinstance(an_adult_fish, FishAdult))
print(isinstance(an_adult_fish, FishLarvae))

True
True
False


# Args and kwargs

In [31]:
class FishLarvae(Fish):
    def __init__(self):
        print("Instantiating a larvae")

In [36]:
a_larvae_fish = FishLarvae(age = 7)
a_larvae_fish.bout()

Instantiating a larvae


AttributeError: 'FishLarvae' object has no attribute 'pos_x'

Of course, one option would be to repeat all arguments in the subclass:

In [65]:
class FishLarvae(Fish):
    def __init__(self, age, start_pos_x=0, start_pos_y=0, max_speed=10):
        super().__init__(age, start_pos_x=start_pos_x, start_pos_y=start_pos_y, max_speed=max_speed)
        print("Instantiating a larvae")

In [66]:
a_larvae_fish = FishLarvae(7)
a_larvae_fish.bout()
a_larvae_fish.print_pos()

Instantiating a larvae
-7 -2


But a much cleaner syntax would be the following:

In [67]:
class FishLarvae(Fish):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print("Instantiating a larvae")

We can still use the arguments stored in args (a list) and kwargs (a dictionary)

In [94]:
class FishLarvae(Fish):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        if args[0] > 9:
            raise ValueError("The fish is too old to be a larvae!")
        
        print("Instantiating a larvae in position ({}, {})".format(kwargs["start_pos_x"], kwargs["start_pos_y"]))
        

In [97]:
a_larvae_fish = FishLarvae(8, start_pos_x=10, start_pos_y=20)
a_larvae_fish.bout()
a_larvae_fish.print_pos()

Instantiating a larvae in position (10, 20)
13 14


In [57]:
def summing(a, b=1):
    print(a+b)

In [58]:
summing(1, 2)

3


# Multiple inheritance

Let's assume we want to keep track of where each fish moves. To do this, we already provide a very neat class to store in a list some quantities:

In [107]:
class DataLogger():
    def __init__(self, what_to_log=[]):
        print("creating DataLogger")
        self.what_to_log = what_to_log
        self.log = {k: [] for k in self.what_to_log}
        
    def update_log(self):
        for k in self.what_to_log:
            self.log[k].append(getattr(self, k))
            
    def reset_log(self):
        self.log = {k: [] for k in self.what_to_log}

Right now, this data logger has nothing interesting to store. But now, let's combine it together with  the fish class!

In [109]:
class LoggedFish(Fish, DataLogger):
    pass

In [110]:
a_logged_fish = LoggedFish(age=10, what_to_log=["pos_x"])

TypeError: __init__() got an unexpected keyword argument 'what_to_log'